# Lab 2 - OpenAI Agents SDK!

2 steps to making an Agent:

1. Create a new class:

`agent = Agent(...)`

2. Call Runner.run

`Runner.run(agent, input)`

For this first part we will explore:

- The System Prompt with instructions
- Runner.run()
- Using LiteLLM to switch models
- Structured Outputs with Pydantic objects

In [2]:
from agents import Agent, Runner
from IPython.display import Markdown, display
from pydantic import BaseModel, Field
import os
from agents.extensions.models.litellm_model import LitellmModel
# load environment variables
from dotenv import load_dotenv
load_dotenv(override=True)


True

In [3]:
autonomous_agent = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model="gpt-4.1-mini")

In [4]:
input = """
You and a partner are contestants on a game show. You're each taken to separate rooms and given a choice:

Cooperate: Choose "Share" — if both of you choose this, you each win $1,000.

Defect: Choose "Steal" — if one steals and the other shares, the stealer gets $2,000 and the sharer gets nothing.

If both steal, you both get nothing.

Do you choose to Steal or Share? Pick one.
"""

In [ ]:
result = await Runner.run(autonomous_agent, input)
display(Markdown(result.final_output))

I choose to Share. Cooperation maximizes our combined gain and builds trust. What do you choose?

[non-fatal] Tracing: request failed: [Errno 8] nodename nor servname provided, or not known
[non-fatal] Tracing: request failed: [Errno 8] nodename nor servname provided, or not known
[non-fatal] Tracing: request failed: [Errno 8] nodename nor servname provided, or not known
[non-fatal] Tracing: max retries reached, giving up on this batch.
[non-fatal] Tracing: request failed: [Errno 8] nodename nor servname provided, or not known
[non-fatal] Tracing: request failed: [Errno 8] nodename nor servname provided, or not known
[non-fatal] Tracing: request failed: [Errno 8] nodename nor servname provided, or not known
[non-fatal] Tracing: max retries reached, giving up on this batch.
[non-fatal] Tracing: request failed: [Errno 8] nodename nor servname provided, or not known
[non-fatal] Tracing: request failed: [Errno 8] nodename nor servname provided, or not known
[non-fatal] Tracing: request failed: [Errno 8] nodename nor servname provided, or not known
[non-fatal] Tracing: max retries reach

Let’s look closely at what changed between your first example (Jokester) and this second one (Autonomous Agent), because it illustrates how Agent behavior is customized in the openai-agents framework.

🧩 1️⃣ The structural difference

| Feature                | `Jokester` Example                                                                     | `Autonomous Agent` Example                                                                         |
| ---------------------- | -------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------- |
| **Agent Definition**   | `Agent(name="Jokester", model="gpt-5-nano")`                                           | `Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model="gpt-4.1-mini")` |
| **User Input**         | `"Tell me a joke about Agentic AI"` (literal string passed directly into `Runner.run`) | `input = """Game theory scenario..."""` (a more complex, multi-line string describing a situation) |
| **Instructions Field** | *none (uses default behavior)*                                                         | `"You are an autonomous agent"` — sets the agent’s internal role or persona                        |
| **Model Used**         | `gpt-5-nano`                                                                           | `gpt-4.1-mini`                                                                                     |
| **Goal**               | Produce a single funny output                                                          | Make a decision under a set of rules (“Steal” vs “Share”)                                          |


## Now let's use LiteLLM to switch up to different models

Here are all the providers:

https://docs.litellm.ai/docs/providers

In [ ]:
model = LitellmModel(model="xai/grok-4", api_key=os.getenv("GROK_API_KEY"))
autonomous_agent = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model=model)
result = await Runner.run(autonomous_agent, input)
display(Markdown(result.final_output))

In [9]:
"claude-3-5-haiku-latest"
model = LitellmModel(model="claude-3-5-haiku-latest", api_key=os.getenv("ANTHROPIC_API_KEY"))
autonomous_agent = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model=model)
result = await Runner.run(autonomous_agent, input)
display(Markdown(result.final_output))

Share.

My reasoning is based on game theory and the concept of cooperative strategies like the "Tit for Tat" approach. In this scenario, mutual cooperation (both choosing "Share") leads to the best collective outcome. By choosing to share, I signal good faith and create the highest probability of both of us winning $1,000.

If I chose to steal, even though it could potentially net me $2,000, it would:
1. Destroy trust
2. Likely provoke retaliation
3. Risk us both getting nothing
4. Be ethically dubious

The cooperative strategy maximizes the likely positive outcome for both parties. It assumes my partner will also recognize the rational choice is to share.

Would you be interested in hearing more about the game theory behind this decision?

In [ ]:
model = LitellmModel(model="deepseek/deepseek-reasoner")
autonomous_agent = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model=model)
result = await Runner.run(autonomous_agent, input)
display(Markdown(result.final_output))

If no api_key argument is passed to LitellmModel, it will find it in your .env; it has code to find the right api_key for the particular model

### 🧩 4️⃣ When you should use it

#### Use LitellmModel when:

You want your agent to use a non-OpenAI model (Anthropic, Mistral, Grok, etc.)

You want to swap providers dynamically without changing your Agent logic.

You want to test or compare multiple LLMs (e.g. Claude vs GPT vs Grok) using the same agent prompt and runner.

In [14]:
"""
        ┌────────────────────────────┐
        │        Runner.run()        │
        └────────────┬───────────────┘
                     │
                     ▼
             ┌────────────────┐
             │     Agent      │
             └───────┬────────┘
                     │
      ┌──────────────┴──────────────┐
      │        Model Interface      │
      ├─────────────────────────────┤
      │  OpenAIModel()   OR         │
      │  LitellmModel()             │
      └──────────────┬──────────────┘
                     │
                     ▼
      ┌────────────────────────────┐
      │ External LLM API (Claude,  │
      │ DeepSeek, GPT-4, Grok, …)  │
      └────────────────────────────┘

"""

'\n        ┌────────────────────────────┐\n        │        Runner.run()        │\n        └────────────┬───────────────┘\n                     │\n                     ▼\n             ┌────────────────┐\n             │     Agent      │\n             └───────┬────────┘\n                     │\n      ┌──────────────┴──────────────┐\n      │        Model Interface      │\n      ├─────────────────────────────┤\n      │  OpenAIModel()   OR         │\n      │  LitellmModel()             │\n      └──────────────┬──────────────┘\n                     │\n                     ▼\n      ┌────────────────────────────┐\n      │ External LLM API (Claude,  │\n      │ DeepSeek, GPT-4, Grok, …)  │\n      └────────────────────────────┘\n\n'

In [18]:
# The famous trolley dilemma

input2 = """
A runaway trolley is heading down a track. Ahead, five people are tied to the tracks and will be killed if the trolley continues.

You are standing next to a lever. If you pull it, the trolley will switch to a different track — but one person is tied to that one.

Do you pull the lever? Choose to pull or not to pull.
"""

## Structured Outputs

In the next cell, we define a Pydantic object.

We will then ask our LLM to generate a response that meets this output schema.

In [19]:
class Decision(BaseModel):
    reasoning: str = Field(description="The rationale for your decision")
    counter_argument: str = Field(description="A counter-argument to the reasoning")
    pull_lever: bool = Field(description="Whether to pull the lever")

In [21]:
autonomous_agent_with_structure = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model="gpt-5-nano", output_type=Decision)
result = await Runner.run(autonomous_agent_with_structure, input2)
decision = result.final_output_as(Decision)
print("Pull lever?", decision.pull_lever)
print("Reasoning:", decision.reasoning)
print("Counter-argument:", decision.counter_argument)


Error getting response: Connection error.. (request_id: None)


APIConnectionError: Connection error.

In [29]:
print(decision.model_dump_json(indent=2))

{
  "reasoning": "Steal (Defect) is the dominant strategy in this one-shot Prisoner's Dilemma. If the other person shares, stealing yields $2,000 instead of $1,000. If the other person steals, stealing yields $0 just as sharing would, so stealing never results in a worse outcome and often yields a better one. Therefore, the optimal choice is Steal.",
  "counter_argument": "A counter-argument is that stealing can destroy trust and lead to mutual loss in expectation, especially if both players reason similarly or in repeated games. In many real-world scenarios, cooperation or binding agreements can lead to higher collective payoff over time, so relying on Steal may be riskier if long-term relationships or reputations matter.",
  "pull_lever": true
}


Field is used in this code is critical to getting reliable results from the Language Model (LLM).

The Field function is recommended and used here as the primary tool to ensure the LLM's output is structured, accurate, and consistently formatted according to the Decision class.

### 1. Why Field is Recommended (For the LLM)

When you use the Decision class as the output_type for the agent, you are essentially asking the LLM to generate a piece of data that perfectly matches that structure. This is called Structured Output.

The Field function acts as the instruction manual for the LLM.

Clarity for the Model: An LLM doesn't natively know what reasoning: str means. However, when you use Field(description="The rationale for your decision"), the LLM sees the specific instructions: "The value you place in the 'reasoning' field must be the rationale for your decision."

Enforcing Quality: Without the descriptive text in Field, the LLM might hallucinate or put brief, unhelpful content into the fields. The description forces the LLM to generate output that adheres to the intent of the field, not just the technical type (str or bool).

Preventing Hallucination: Clear descriptions drastically reduce the chances of the LLM placing irrelevant or incorrectly formatted text into your structured output, thereby improving the performance of your entire agent system.

### 2. How Field Is Used in This Code (Metadata and Schema)

Field is being used here to attach metadata to the standard Pydantic data model.

A. Defining the Pydantic Model
Pydantic is a library used for data validation and serialization. The BaseModel defines the expected shape of your data:

In [15]:
class Decision(BaseModel):
    # Field names and Python types are defined here
    reasoning: str
    counter_argument: str
    pull_lever: bool

### B. Generating the JSON Schema
When you pass the Decision class to the agent's runner, the system framework (LiteLLM, LangChain, etc.) automatically converts this Python class into a JSON Schema.

Crucially, the text you put inside the description argument of Field is placed directly into the JSON Schema.

Example of the JSON Schema sent to the LLM:

In [16]:
"""
{
  "type": "object",
  "properties": {
    "reasoning": {
      "type": "string",
      // This is where the Field description goes!
      "description": "The rationale for your decision"
    },
    "pull_lever": {
      "type": "boolean",
      "description": "Whether to pull the lever"
    }
  },
  // ... rest of the schema
}
"""

'\n{\n  "type": "object",\n  "properties": {\n    "reasoning": {\n      "type": "string",\n      // This is where the Field description goes!\n      "description": "The rationale for your decision" \n    },\n    "pull_lever": {\n      "type": "boolean",\n      "description": "Whether to pull the lever"\n    }\n  },\n  // ... rest of the schema\n}\n'

The LLM is then given this full JSON Schema and instructed to produce a JSON object that matches it. The description ensures the LLM understands its role for each specific piece of data it is creating.